<a href="https://colab.research.google.com/github/parseny/nlp_course/blob/2023/week05_transfer/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [1]:
%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets

### Load data and model

In [2]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

Repo card metadata block was not found. Setting CardData to empty.




Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [12]:
qqp['train']

Dataset({
    features: ['text1', 'text2', 'label', 'idx', 'label_text'],
    num_rows: 363846
})

In [3]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

### Tokenize the data

In [4]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [5]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (1 point)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [6]:
# for batch in val_loader:
#      break  # here be your training code
# print("Sample batch:", batch)

# with torch.no_grad():
#   predicted = model(
#       input_ids=batch['input_ids'],
#       attention_mask=batch['attention_mask'],
#       token_type_ids=batch['token_type_ids']
#   )

# print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [12]:
model = model.to(device)

In [13]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=16, shuffle=False, collate_fn=transformers.default_data_collator, num_workers=5
)

In [14]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm


labels_pred = []
labels_true = []

for batch in tqdm(val_loader):
    b = {k: v.to(device) for k, v in batch.items()}
    predicted = model(
      input_ids=b['input_ids'],
      attention_mask=b['attention_mask'],
      token_type_ids=b['token_type_ids']
    )
    preds = torch.argmax(predicted.logits, dim=-1).data
    trues = b['labels'].data
    labels_pred.extend(preds.cpu().numpy())
    labels_true.extend(trues.cpu().numpy())

accuracy = accuracy_score(labels_true, labels_pred)

100%|██████████| 2527/2527 [04:45<00:00,  8.84it/s]


In [17]:
assert 0.9 < accuracy < 0.91
accuracy

0.9083848627256987

### Task 2: train the model (4 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

In [3]:
!pip install transformers[sentencepiece]

In [4]:
model_name = 'microsoft/deberta-v3-base'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['pooler.dense.weight', 'classifier.bias', 'pooler.dense.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

In [8]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[1, 577, 269, 262, 432, 265, 266, 5291, 1234, 302, 5047, 274, 3443, 290, 451, 2056, 302, 2, 2597, 67 ...


In [10]:
model = model.to(device)

In [11]:
train_set = qqp_preprocessed['train']
val_set = qqp_preprocessed['validation']

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=20, shuffle=True, collate_fn=transformers.default_data_collator, num_workers=2
)

In [ ]:
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=8, shuffle=False, collate_fn=transformers.default_data_collator, num_workers=2
)

In [13]:
!git clone https://github.com/huggingface/transformers.git


Cloning into 'transformers'...
remote: Enumerating objects: 166346, done.
remote: Counting objects: 100% (2551/2551), done.
remote: Compressing objects: 100% (1229/1229), done.
remote: Total 166346 (delta 1688), reused 1830 (delta 1176), pack-reused 163795
Receiving objects: 100% (166346/166346), 167.74 MiB | 14.87 MiB/s, done.
Resolving deltas: 100% (125095/125095), done.


In [14]:
cd transformers/examples/pytorch/text-classification

/content/transformers/examples/pytorch/text-classification


In [16]:
train_path = "/content/qqp_train.csv"
validation_path = "/content/qqp_validation.csv"

qqp["train"].to_csv(train_path)
qqp["validation"].to_csv(validation_path)


Creating CSV from Arrow format:   0%|          | 0/364 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/41 [00:00<?, ?ba/s]

5772427

In [17]:
!python run_classification.py \
  --model_name_or_path microsoft/deberta-v3-base \
  --dataset_name 'SetFit/qqp' \
  --shuffle_train_dataset \
  --metric_name accuracy \
  --do_train \
  --do_eval \
  --train_file train_path \
  --validation_file validation_path \
  --max_seq_length 128 \
  --per_device_train_batch_size 16 \
  --learning_rate 2e-5 \
  --num_train_epochs 2 \
  --output_dir /content/deberta-qqp-output/ \
  --overwrite_output_dir


Traceback (most recent call last):
  File "/content/transformers/examples/pytorch/text-classification/run_classification.py", line 28, in <module>
    import evaluate
ModuleNotFoundError: No module named 'evaluate'


### Task 3: try the full pipeline (1 point)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.